## 📦 Setup & Configuration

In [1]:
# Включаем DEBUG логирование
import os
os.environ['LOG_LEVEL'] = 'DEBUG'

# Загружаем .env
from app.config import ensure_dotenv
ensure_dotenv()

# Настраиваем логирование
from app.logging_config import configure_logging
configure_logging()

print('✅ Logging configured')

✅ Logging configured



In [2]:
# Проверяем переменные окружения для агентов
env_vars = [
    'GIGACHAT_CREDENTIALS',
    'GIGACHAT_SCOPE', 
    'GIGACHAT_MODEL',
    'EMBEDDINGS_MODEL',
    'API_GEO',
    'API_SITE',
    'REGION_ID',
    'LANGSMITH_TRACING',
]

print('🔧 Environment Variables:')
print('=' * 50)
for var in env_vars:
    val = os.getenv(var, 'NOT SET')
    if 'CREDENTIALS' in var or 'API_KEY' in var:
        val = f'{val[:20]}...' if len(val) > 20 else val
    print(f'{var}: {val}')

🔧 Environment Variables:
GIGACHAT_CREDENTIALS: MDE5YWNmM2YtMjFiOC03...
GIGACHAT_SCOPE: GIGACHAT_API_PERS

GIGACHAT_CREDENTIALS: MDE5YWNmM2YtMjFiOC03...
GIGACHAT_SCOPE: GIGACHAT_API_PERS
GIGACHAT_MODEL: GigaChat-2
EMBEDDINGS_MODEL: EmbeddingsGigaR
API_GEO: https://yazzh-geo.gate.petersburg.ru
API_SITE: https://yazzh.gate.petersburg.ruGIGACHAT_MODEL: GigaChat-2
EMBEDDINGS_MODEL: EmbeddingsGigaR
API_GEO: https://yazzh-geo.gate.petersburg.ru
API_SITE: https://yazzh.gate.petersburg.ru
REGION_ID: 78
LANGSMITH_TRACING: true

REGION_ID: 78
LANGSMITH_TRACING: true


---
## 🔤 LLM Configuration

In [3]:
# Создаём LLM
from app.agent.llm import get_llm

llm = get_llm()
print(f'✅ LLM created: {type(llm).__name__}')
print(f'   Model: {llm.model if hasattr(llm, "model") else "N/A"}')

f:\dropped_out\prog.proj\sber_agent_hackathon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ LLM created: GigaChat
   Model: GigaChat-2-Max

   Model: GigaChat-2-Max


In [4]:
# Тестовый вызов LLM
from langchain_core.messages import HumanMessage

test_message = HumanMessage(content='Привет! Как дела?')
response = llm.invoke([test_message])

print('🔤 LLM Test:')
print(f'   Input: {test_message.content}')
print(f'   Output: {response.content}')

OAUTH UPDATE TOKEN
Tracing control thread func compress parallel called
Tracing control thread func compress parallel called
Sending multipart request to https://api.smith.langchain.com with context: trace=b679bcc5-c215-4830-a568-6572dc608c16,id=b679bcc5-c215-4830-a568-6572dc608c16
Sending multipart request to https://api.smith.langchain.com with context: trace=b679bcc5-c215-4830-a568-6572dc608c16,id=b679bcc5-c215-4830-a568-6572dc608c16
🔤 LLM Test:
   Input: Привет! Как дела?
   Output: Привет! У меня всё отлично — готов общаться, обсуждать интересные темы, решать задачи или просто болтать обо всём подряд. А ты как себя чувствуешь сегодня?
🔤 LLM Test:
   Input: Привет! Как дела?
   Output: Привет! У меня всё отлично — готов общаться, обсуждать интересные темы, решать задачи или просто болтать обо всём подряд. А ты как себя чувствуешь сегодня?


---
## 🛡️ Toxicity Filter Testing

In [5]:
# Загружаем toxicity filter
from app.services.toxicity import get_toxicity_filter, ToxicityLevel

toxicity_filter = get_toxicity_filter()
print('✅ Toxicity filter loaded')

✅ Toxicity filter loaded



In [6]:
# Sample тексты для тестирования
TOXICITY_SAMPLES = [
    ('Как получить СНИЛС?', False, 'Обычный вопрос'),
    ('Спасибо за помощь!', False, 'Благодарность'),
    ('Блин, ничего не работает', True, 'Лёгкое недовольство'),
    ('Какого хрена?!', True, 'Грубость'),
    ('Это полный бред', True, 'Негатив'),
]

print('🛡️ Toxicity Filter Tests:')
print('=' * 60)

for text, expected_toxic, description in TOXICITY_SAMPLES:
    result = toxicity_filter.check(text)
    status = '✅' if result.should_block == expected_toxic else '❌'
    print(f'\n{status} "{text}"')
    print(f'   Description: {description}')
    print(f'   Level: {result.level.value}')
    print(f'   Should Block: {result.should_block} (expected: {expected_toxic})')
    print(f'   Confidence: {result.confidence:.2f}')
    if result.matched_patterns:
        print(f'   Matched: {result.matched_patterns[:3]}')

🛡️ Toxicity Filter Tests:

✅ "Как получить СНИЛС?"
   Description: Обычный вопрос
   Level: safe

✅ "Как получить СНИЛС?"
   Description: Обычный вопрос
   Level: safe
   Should Block: False (expected: False)
   Should Block: False (expected: False)
   Confidence: 1.00

✅ "Спасибо за помощь!"
   Description: Благодарность
   Level: safe
   Should Block: False (expected: False)
   Confidence: 1.00

❌ "Блин, ничего не работает"
   Description: Лёгкое недовольство
   Confidence: 1.00

✅ "Спасибо за помощь!"
   Description: Благодарность
   Level: safe
   Should Block: False (expected: False)
   Confidence: 1.00

❌ "Блин, ничего не работает"
   Description: Лёгкое недовольство
   Level: low
   Should Block: False (expected: True)

   Level: low
   Should Block: False (expected: True)
   Confidence: 0.70
   Matched: ['\\bблин\\b']

❌ "Какого хрена?!"
   Description: Грубость   Confidence: 0.70
   Matched: ['\\bблин\\b']

❌ "Какого хрена?!"
   Description: Грубость
   Level: safe
   Should B

In [7]:
# Получаем ответ для токсичного сообщения
toxic_text = 'Это полный бред'
result = toxicity_filter.check(toxic_text)
response = toxicity_filter.get_response(result)

print('📝 Toxicity Response:')
print(f'   Input: {toxic_text}')
print(f'   Response: {response}')

📝 Toxicity Response:
   Input: Это полный бред
   Response: None

   Input: Это полный бред
   Response: None


---
## 🔧 Tools Testing

In [8]:
# Загружаем доступные tools
from app.tools import ALL_TOOLS

print(f'✅ Loaded {len(ALL_TOOLS)} tools:')
for tool in ALL_TOOLS:
    print(f'   - {tool.name}: {tool.description[:60]}...')

✅ Loaded 4 tools:
   - find_nearest_mfc_tool: Найти ближайший МФЦ (Многофункциональный центр) по адресу по...
   - get_pensioner_categories_tool: Получить список категорий услуг для пенсионеров

Используй э...
   - get_pensioner_services_tool: Найти услуги для пенсионеров по району и категориям.

Исполь...
   - search_city_services: Поиск информации о государственных услугах Санкт-Петербурга....

   - find_nearest_mfc_tool: Найти ближайший МФЦ (Многофункциональный центр) по адресу по...
   - get_pensioner_categories_tool: Получить список категорий услуг для пенсионеров

Используй э...
   - get_pensioner_services_tool: Найти услуги для пенсионеров по району и категориям.

Исполь...
   - search_city_services: Поиск информации о государственных услугах Санкт-Петербурга....


In [9]:
# Тестируем tool напрямую (пример с geocode)
# Раскомментируй для тестирования:

# from app.tools.city_api import geocode_address
# 
# result = geocode_address.invoke({'address': 'Невский проспект 1'})
# print('📍 Geocode Result:')
# print(result)

---
## 🔀 Agent State Testing

In [10]:
# Импортируем состояния агентов
from app.agent.state import (
    AgentState,
    get_last_user_message,
    create_ai_response,
    create_error_response,
)
from langchain_core.messages import HumanMessage, AIMessage

print('✅ Agent states imported')

✅ Agent states imported



In [11]:
# Тестируем извлечение сообщения
# Разные форматы content (как от LangGraph Studio)

test_states = [
    # Формат 1: простая строка
    {'messages': [HumanMessage(content='Привет')]},
    
    # Формат 2: list[dict] от Studio
    {'messages': [HumanMessage(content=[{'type': 'text', 'text': 'Как дела?'}])]},
    
    # Формат 3: пустой
    {'messages': []},
]

print('📝 get_last_user_message() Tests:')
print('=' * 50)

for i, state in enumerate(test_states, 1):
    result = get_last_user_message(state)
    print(f'{i}. Input: {state["messages"]}')
    print(f'   Output: "{result}"')
    print()

📝 get_last_user_message() Tests:
1. Input: [HumanMessage(content='Привет', additional_kwargs={}, response_metadata={})]
   Output: "Привет"


1. Input: [HumanMessage(content='Привет', additional_kwargs={}, response_metadata={})]
   Output: "Привет"

2. Input: [HumanMessage(content=[{'type': 'text', 'text': 'Как дела?'}], additional_kwargs={}, response_metadata={})]
   Output: "Как дела?"
2. Input: [HumanMessage(content=[{'type': 'text', 'text': 'Как дела?'}], additional_kwargs={}, response_metadata={})]
   Output: "Как дела?"

3. Input: []
   Output: ""


3. Input: []
   Output: ""



In [12]:
# Тестируем создание ответов
response = create_ai_response('Это тестовый ответ')
print('📝 create_ai_response():')
print(f'   {response}')

error_response = create_error_response(
    Exception('Test error'),
    fallback_message='Произошла ошибка'
)
print('\n📝 create_error_response():')
print(f'   {error_response}')

📝 create_ai_response():
   {'messages': [AIMessage(content='Это тестовый ответ', additional_kwargs={}, response_metadata={})], 'final_response': 'Это тестовый ответ'}
19:25:05 [error   ] agent_error                    app=city-assistant error='Test error'

📝 create_error_response():

   {'messages': [AIMessage(content='Это тестовый ответ', additional_kwargs={}, response_metadata={})], 'final_response': 'Это тестовый ответ'}
19:25:05 [error   ] agent_error                    app=city-assistant error='Test error'

📝 create_error_response():
   {'messages': [AIMessage(content='Произошла ошибка', additional_kwargs={}, response_metadata={})], 'final_response': 'Произошла ошибка', 'metadata': {'error': 'Test error', 'error_type': 'Exception'}}
   {'messages': [AIMessage(content='Произошла ошибка', additional_kwargs={}, response_metadata={})], 'final_response': 'Произошла ошибка', 'metadata': {'error': 'Test error', 'error_type': 'Exception'}}


---
## 🔄 Hybrid Agent Testing

In [13]:
# Загружаем Hybrid Agent
from app.agent.hybrid import get_hybrid_graph

hybrid_graph = get_hybrid_graph()
print('✅ Hybrid Agent loaded')
print(f'   Nodes: {list(hybrid_graph.nodes.keys())}')

19:25:05 [info    ] hybrid_graph_build_start        [info    ] hybrid_graph_build_start       app=city-assistant with_checkpointer=app=city-assistant with_checkpointer=False
False
19:25:05 [info    ] 19:25:05 [info    ] hybrid_graph_build_complete    app=city-assistant nodeshybrid_graph_build_complete    app=city-assistant nodes=['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'tool_agent', 'rag_search', 'conversation', 'generate_response'] with_checkpointer=False
✅ Hybrid Agent loaded
=['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'tool_agent', 'rag_search', 'conversation', 'generate_response'] with_checkpointer=False
✅ Hybrid Agent loaded
   Nodes: ['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'tool_agent', 'rag_search', 'conversation', 'generate_response']
   Nodes: ['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'tool_agent', 'rag_search', 'conversation', 'generate_response']


In [14]:
# Sample запросы для тестирования
HYBRID_SAMPLES = [
    'Привет, как дела?',
    'Как получить СНИЛС?',
    'Какие документы нужны для загранпаспорта?',
]

print('📝 Hybrid Agent Sample Queries:')
for i, q in enumerate(HYBRID_SAMPLES, 1):
    print(f'   {i}. {q}')

📝 Hybrid Agent Sample Queries:
   1. Привет, как дела?
   2. Как получить СНИЛС?
   3. Какие документы нужны для загранпаспорта?

   1. Привет, как дела?
   2. Как получить СНИЛС?
   3. Какие документы нужны для загранпаспорта?


In [15]:
# Тестируем Hybrid Agent
from langchain_core.messages import HumanMessage

query = HYBRID_SAMPLES[1]
print(f'🤖 Hybrid Agent Test: "{query}"')
print('=' * 60)

initial_state = {
    'messages': [HumanMessage(content=query)],
    'is_toxic': False,
    'toxicity_response': None,
    'intent': '',
    'intent_confidence': 0.0,
    'tool_result': None,
    'final_response': None,
    'metadata': {},
}

# Запускаем агент
result = hybrid_graph.invoke(initial_state)

print('\n📊 Result:')
print(f'   is_toxic: {result.get("is_toxic")}')
print(f'   Messages count: {len(result.get("messages", []))}')

# Последнее сообщение (ответ агента)
if result.get('messages'):
    last_msg = result['messages'][-1]
    print(f'\n💬 Agent Response:')
    print(f'   Type: {type(last_msg).__name__}')
    content = last_msg.content if hasattr(last_msg, 'content') else str(last_msg)
    print(f'   Content: {content[:500]}...' if len(str(content)) > 500 else f'   Content: {content}')

🤖 Hybrid Agent Test: "Как получить СНИЛС?"

19:25:05 [info    ] hybrid_node                    app=city-assistant node=check_toxicity ============================================================
19:25:05 [info    ] hybrid_node                    app=city-assistant node=check_toxicity query_length=19
19:25:05 [info    ] hybrid_node                    appquery_length=19
19:25:05 [info    ] hybrid_node                    app=city-assistant node=classify_intent query='как получить снилс?'
=city-assistant node=classify_intent query='как получить снилс?'
19:25:05 [info    ] intent_classified              app19:25:05 [info    ] intent_classified              app=city-assistant intent==city-assistant intent=rag_search method=keywords
19:25:05 [info    ] hybrid_node                   rag_search method=keywords
19:25:05 [info    ] hybrid_node                    app=city-assistant node=rag_search query='Как получить СНИЛС?'
19:25:05 app=city-assistant node=rag_search query='Как получить СНИЛС?'
1

In [16]:
# Тестируем с токсичным запросом
toxic_query = 'Да что за ерунда такая?!'
print(f'🤖 Hybrid Agent Toxicity Test: "{toxic_query}"')
print('=' * 60)

toxic_state = {
    'messages': [HumanMessage(content=toxic_query)],
    'is_toxic': False,
    'toxicity_response': None,
    'intent': '',
    'intent_confidence': 0.0,
    'tool_result': None,
    'final_response': None,
    'metadata': {},
}

result = hybrid_graph.invoke(toxic_state)

print(f'\n📊 Result:')
print(f'   is_toxic: {result.get("is_toxic")}')
print(f'   toxicity_response: {result.get("toxicity_response")}')

🤖 Hybrid Agent Toxicity Test: "Да что за ерунда такая?!"
19:25:08 [info    ] 
19:25:08 [info    ] hybrid_node                    app=hybrid_node                    app=city-assistant node=check_toxicity query_length=24
city-assistant node=check_toxicity query_length=24
19:25:08 [info    ] hybrid_node                    app=city-assistant19:25:08 [info    ] hybrid_node                    app=city-assistant node=classify_intent node=classify_intent query='да что за ерунда такая?!'
19:25:08 [info    ]  query='да что за ерунда такая?!'
19:25:08 [info    ] intent_classified              app=city-assistant intent=rag_search methodintent_classified              app=city-assistant intent=rag_search method=keywords
19:25:08 [info    ] hybrid_node                    app=keywords
19:25:08 [info    ] hybrid_node                    app=city-assistant node=rag_search query='Да что за ерунда такая?!'
19:25:08 [info    ] graph_invoke_start             app=city-assistant=city-assistant node=rag_search 

---
## 🎯 Supervisor Agent Testing

In [17]:
# Загружаем Supervisor Agent
from app.agent.supervisor import get_supervisor_graph

supervisor_graph = get_supervisor_graph()
print('✅ Supervisor Agent loaded')
print(f'   Nodes: {list(supervisor_graph.nodes.keys())}')

19:25:10 [info    ] supervisor_graph_build_start   app=city-assistant with_checkpointer=False
19:25:10 [info    ] supervisor_graph_build_start   app=city-assistant with_checkpointer=False
19:25:10 [info    ] supervisor_graph_build_complete app=city-assistant nodes=['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'api_handler', 'rag_search', 'conversation', 'generate_response']  [info    ] supervisor_graph_build_complete app=city-assistant nodes=['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'api_handler', 'rag_search', 'conversation', 'generate_response'] with_checkpointer=False
✅ Supervisor Agent loaded
   Nodes: ['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'api_handler', 'rag_search', 'conversation', 'generate_response']with_checkpointer=False
✅ Supervisor Agent loaded
   Nodes: ['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'api_handler', 'rag_search', 'conversation', 'generate_response']



In [18]:
# Sample запросы для разных интентов
SUPERVISOR_SAMPLES = [
    ('Привет!', 'greeting'),
    ('Как оформить паспорт?', 'rag_query'),
    ('Найди адрес Невский 1', 'api_call'),
    ('Спасибо, до свидания', 'farewell'),
]

print('📝 Supervisor Agent Sample Queries:')
for query, expected_intent in SUPERVISOR_SAMPLES:
    print(f'   "{query}" → {expected_intent}')

📝 Supervisor Agent Sample Queries:
   "Привет!" → greeting
   "Как оформить паспорт?" → rag_query
   "Найди адрес Невский 1" → api_call
   "Спасибо, до свидания" → farewell

   "Привет!" → greeting
   "Как оформить паспорт?" → rag_query
   "Найди адрес Невский 1" → api_call
   "Спасибо, до свидания" → farewell


In [19]:
# Тестируем Supervisor Agent
query, expected_intent = SUPERVISOR_SAMPLES[1]  # rag_query

print(f'🎯 Supervisor Test: "{query}"')
print(f'   Expected intent: {expected_intent}')
print('=' * 60)

initial_state = {
    'messages': [HumanMessage(content=query)],
    'is_toxic': False,
    'toxicity_response': None,
    'intent': '',
    'intent_confidence': 0.0,
    'tool_result': None,
    'final_response': None,
    'metadata': {},
    # Supervisor-specific
    'next_agent': None,
    'rag_context': None,
}

result = supervisor_graph.invoke(initial_state)

print('\n📊 Result:')
print(f'   intent: {result.get("intent")}')
print(f'   intent_confidence: {result.get("intent_confidence")}')
print(f'   next_agent: {result.get("next_agent")}')

# Ответ
if result.get('messages'):
    last_msg = result['messages'][-1]
    content = last_msg.content if hasattr(last_msg, 'content') else str(last_msg)
    print(f'\n💬 Response:')
    print(f'   {content[:300]}...' if len(str(content)) > 300 else f'   {content}')

🎯 Supervisor Test: "Как оформить паспорт?"
   Expected intent: rag_query
19:25:10 [info    ] supervisor_node                
   Expected intent: rag_query
19:25:10 [info    ] supervisor_node                app=city-assistant node=check_toxicity query_length=21
app=city-assistant node=check_toxicity query_length=21
19:25:10 [debug   19:25:10 [debug   ] toxicity_passed                app=city-assistant
] toxicity_passed                app=city-assistant
OAUTH UPDATE TOKEN
OAUTH UPDATE TOKEN
19:25:11 [info    ] intent_classified              app=city-assistant intent=rag_search 19:25:11 [info    ] intent_classified              app=city-assistant intent=rag_search llm_response=rag_search method=llm
19:25:11llm_response=rag_search method=llm
19:25:11 [info    ] supervisor_node                app=city-assistant node=rag_search query= [info    ] supervisor_node                app=city-assistant node=rag_search query='Как оформить паспорт?'
19:25:11 [info    ] graph_invoke_start             a

---
## 📜 Prompts Inspection

In [20]:
# Смотрим системные промпты агентов
# Раскомментируй модули которые хочешь проверить

print('📜 Agent Prompts:')
print('=' * 60)

# Hybrid Agent prompt
try:
    from app.agent.hybrid import SYSTEM_PROMPT as HYBRID_PROMPT
    print('\n🔄 Hybrid Agent System Prompt:')
    print('-' * 40)
    print(HYBRID_PROMPT[:500] + '...' if len(HYBRID_PROMPT) > 500 else HYBRID_PROMPT)
except ImportError:
    print('   Hybrid prompt not exported')

# RAG prompts
try:
    from app.rag.enhancers import QueryRewriter
    rewriter = QueryRewriter()
    print('\n📝 Query Rewriter Prompt:')
    print('-' * 40)
    # Если есть доступ к промпту
except Exception as e:
    print(f'   Could not load: {e}')

📜 Agent Prompts:

   Hybrid prompt not exported

📝 Query Rewriter Prompt:
----------------------------------------
   Hybrid prompt not exported

📝 Query Rewriter Prompt:
----------------------------------------


---
## 🧪 LangGraph Studio Format Testing

Тестирование форматов сообщений от LangGraph Studio

In [21]:
# Симулируем формат от LangGraph Studio
from langchain_core.messages import HumanMessage

# Studio отправляет content как list[dict]
studio_message = HumanMessage(
    content=[{'type': 'text', 'text': 'Как получить паспорт?'}]
)

studio_state = {
    'messages': [studio_message],
    'is_toxic': False,
    'toxicity_response': None,
    'intent': '',
    'intent_confidence': 0.0,
    'tool_result': None,
    'final_response': None,
    'metadata': {},
}

# Извлекаем текст
extracted = get_last_user_message(studio_state)
print('🖥️ LangGraph Studio Format Test:')
print(f'   Raw content: {studio_message.content}')
print(f'   Extracted text: "{extracted}"')

🖥️ LangGraph Studio Format Test:
   Raw content: [{'type': 'text', 'text': 'Как получить паспорт?'}]
   Extracted text: "Как получить паспорт?"

   Raw content: [{'type': 'text', 'text': 'Как получить паспорт?'}]
   Extracted text: "Как получить паспорт?"


In [22]:
# Тестируем агент с Studio-форматом
print('🖥️ Hybrid Agent with Studio Format:')
print('=' * 60)

result = hybrid_graph.invoke(studio_state)

print(f'   is_toxic: {result.get("is_toxic")}')
print(f'   Messages: {len(result.get("messages", []))}')

if result.get('messages'):
    last = result['messages'][-1]
    print(f'\n💬 Response: {last.content[:200]}...' if hasattr(last, 'content') else str(last)[:200])

🖥️ Hybrid Agent with Studio Format:
19:25:13 [info    ] hybrid_node                    app=city-assistant node=check_toxicity query_length=21
19:25:13 [info    ] hybrid_node                    app=city-assistant node=classify_intent 
19:25:13 [info    ] hybrid_node                    app=city-assistant node=check_toxicity query_length=21
19:25:13 [info    ] hybrid_node                    app=city-assistant node=classify_intent query='как получить паспорт?'
19:25:13 [info    ] intent_classified              app=city-assistant intent=rag_search method=keywords
19:25:13 [info    ] hybrid_node                    app=city-assistantquery='как получить паспорт?'
19:25:13 [info    ] intent_classified              app=city-assistant intent=rag_search method=keywords
19:25:13 [info    ] hybrid_node                    app=city-assistant node=rag_search query='Как получить паспорт?'
19:25:13 [info    ] graph_invoke_start             app=city-assistant k=5 min_relevant=3 query='Как получить паспорт

---
## 📊 Batch Agent Testing

In [23]:
# Batch тестирование Hybrid Agent
all_samples = [
    'Привет!',
    'Как получить СНИЛС?',
    'Какие льготы есть для пенсионеров?',
    'Спасибо за помощь!',
]

print('📊 Batch Hybrid Agent Tests')
print('=' * 60)

for query in all_samples:
    state = {
        'messages': [HumanMessage(content=query)],
        'is_toxic': False,
        'toxicity_response': None,
        'intent': '',
        'intent_confidence': 0.0,
        'tool_result': None,
        'final_response': None,
        'metadata': {},
    }
    
    try:
        result = hybrid_graph.invoke(state)
        last_msg = result['messages'][-1] if result.get('messages') else None
        response = last_msg.content[:100] if last_msg and hasattr(last_msg, 'content') else 'N/A'
        print(f'\n🔍 "{query}"')
        print(f'   Response: {response}...')
    except Exception as e:
        print(f'\n❌ "{query}"')
        print(f'   Error: {e}')

📊 Batch Hybrid Agent Tests
19:25:15
19:25:15 [info    ] hybrid_node                    app= [info    ] hybrid_node                    app=city-assistant node=check_toxicity query_length=7
city-assistant node=check_toxicity query_length=7
19:25:15 [info    19:25:15 [info    ] hybrid_node                    app=city-assistant node=] hybrid_node                    app=city-assistant node=classify_intent query=привет!
19:25:15 [info    ] intent_classified              app=city-assistant intent=conversation method=keywords
19:25:15 [classify_intent query=привет!
19:25:15 [info    ] intent_classified              app=city-assistant intent=conversation method=keywords
19:25:15 [info    ] hybrid_node                    app=city-assistant node=conversation query=Привет!
info    ] hybrid_node                    app=city-assistant node=conversation query=Привет!
OAUTH UPDATE TOKEN
OAUTH UPDATE TOKEN
Sending compressed multipart request with context: trace=d27d44ef-14b8-4391-a3fb-22813f204111,id=9

---
## 🔗 Full Pipeline Test (graphs.py exports)

In [24]:
# Проверяем экспорты из graphs.py (для langgraph.json)
from app.graphs import hybrid, supervisor, rag

# Это функции-фабрики, вызываем их для получения графов
hybrid_g = hybrid()
supervisor_g = supervisor()
rag_g = rag()

print('✅ All graphs exported from app.graphs:')
print(f'   hybrid: {type(hybrid_g).__name__}')
print(f'   supervisor: {type(supervisor_g).__name__}')
print(f'   rag: {type(rag_g).__name__}')

19:25:22 [info    ]  [info    ] graph_factory_hybrid           app=city-assistant config_present=False
graph_factory_hybrid           app=city-assistant config_present=False
19:25:22 [info    ] hybrid_graph_build_start       app=city-assistant with_checkpointer=False
19:25:22 [info    ] hybrid_graph_build_start       app=city-assistant with_checkpointer=False
19:25:22 [info    ] hybrid_graph_build_complete    app19:25:22 [info    ] hybrid_graph_build_complete    app=city-assistant nodes=['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'tool_agent', 'rag_search', 'conversation', 'generate_response'] with_checkpointer=False=city-assistant nodes=['__start__', 'check_toxicity', 'toxic_response', 'classify_intent', 'tool_agent', 'rag_search', 'conversation', 'generate_response'] with_checkpointer=False
19:25:22 [info    ] graph_factory_supervisor       app=city-assistant config_present=False
19:25:22 [info    ] graph_factory_supervisor       app=city-assistant config_pre

In [25]:
# Финальный интеграционный тест
print('🎯 Final Integration Test')
print('=' * 60)

test_query = 'Какие документы нужны для регистрации ребёнка?'

state = {
    'messages': [HumanMessage(content=test_query)],
    'is_toxic': False,
    'toxicity_response': None,
    'intent': '',
    'intent_confidence': 0.0,
    'tool_result': None,
    'final_response': None,
    'metadata': {},
}

print(f'Query: "{test_query}"')
print()

result = hybrid_graph.invoke(state)

print('Result:')
print(f'  is_toxic: {result.get("is_toxic")}')
print(f'  messages: {len(result.get("messages", []))}')

if result.get('messages'):
    last = result['messages'][-1]
    if hasattr(last, 'content'):
        print(f'\n💬 Final Response:')
        print(last.content)

🎯 Final Integration Test
Query: "Какие документы нужны для регистрации ребёнка?"

19:25:22 [info    ] hybrid_node                    app
Query: "Какие документы нужны для регистрации ребёнка?"

19:25:22 [info    ] hybrid_node                    app=city-assistant node=check_toxicity query_length=46
19:25:22 [info    =city-assistant node=check_toxicity query_length=46
19:25:22 [info    ] hybrid_node                    app=city-assistant node] hybrid_node                    app=city-assistant node=classify_intent =classify_intent query='какие документы нужны для регистрации ребёнка?'query='какие документы нужны для регистрации ребёнка?'
19:25:22 [info    ] intent_classified              app=city-assistant intent=rag_search method=keywords
19:25:22 [info    ] intent_classified              app=city-assistant intent=rag_search method=keywords
19:25:22 [info    ] hybrid_node                    app=city-assistant node=rag_search query='Какие документы нужны для регистрации ребёнка?'
19:25:22

Sending compressed multipart request with context: trace=95140537-1189-4d30-bee9-34f3714aafe7,id=916835f3-eb4a-45c4-869b-2c943d510730; trace=95140537-1189-4d30-bee9-34f3714aafe7,id=895592f3-6141-4063-bcdd-7656ffccb403; trace=95140537-1189-4d30-bee9-34f3714aafe7,id=895592f3-6141-4063-bcdd-7656ffccb403; trace=95140537-1189-4d30-bee9-34f3714aafe7,id=f4edebc1-7df6-4e67-a1b9-fcf938ea06cb; trace=95140537-1189-4d30-bee9-34f3714aafe7,id=40b8e92c-5373-4b6d-8198-57ef056c212e; trace=95140537-1189-4d30-bee9-34f3714aafe7,id=7819838b-3804-4b13-8eec-4a66679f4471; trace=95140537-1189-4d30-bee9-34f3714aafe7,id=7819838b-3804-4b13-8eec-4a66679f4471; trace=95140537-1189-4d30-bee9-34f3714aafe7,id=6249a563-be5a-4ccd-9e87-4fc8c0a0d047; trace=95140537-1189-4d30-bee9-34f3714aafe7,id=73e038f3-20d7-4727-a12a-40dcc1ba3878; trace=95140537-1189-4d30-bee9-34f3714aafe7,id=d7fd6685-1d0b-4a01-883d-fe9408c53489; trace=95140537-1189-4d30-bee9-34f3714aafe7,id=d7fd6685-1d0b-4a01-883d-fe9408c53489; trace=95140537-1189-4d30-